# From Linear Regression to Hierarchical Models

In [ ]:
import numpy as np
import pymc as pm
import arviz as az
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import matplotlib.pyplot as plt
import warnings
import xarray as xr

# Visual style
az.style.use('arviz-doc')
pio.templates.default = 'plotly_white'

# Plotly defaults
px.defaults.template = 'plotly_white'
px.defaults.width = 900
px.defaults.height = 500
_base = pio.templates['plotly_white']
_tmpl = go.layout.Template(_base)
_tmpl.layout.hovermode = 'x unified'
pio.templates['hoverx'] = _tmpl
pio.templates.default = 'plotly_white+hoverx'

# Reproducibility
RANDOM_SEED = 20090425
RNG = np.random.default_rng(RANDOM_SEED)

warnings.filterwarnings("ignore", module="mkl_fft")
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Generalized Linear Models (GLMs)

In the previous section, we worked with linear regression where the outcome variable was continuous and could be reasonably modeled with a normal distribution. However, many real-world outcomes don't fit this comfortable framework. Consider these common scenarios:

- **Count data**: Number of accidents at an intersection, disease cases in a population, customer complaints per day, goals scored in a match
- **Binary outcomes**: Customer churn (yes/no), disease presence (positive/negative), loan default (yes/no), email clicked (yes/no)
- **Proportions**: Market share, exam pass rates, survey response rates, survival probabilities
- **Strictly positive continuous**: Waiting times, insurance claim amounts, reaction times, rainfall amounts
- **Ordinal data**: Customer satisfaction (1-5 stars), pain levels, educational attainment levels

Trying to force these into a linear regression framework with normal errors often fails spectacularly:
- Predictions can be nonsensical (negative counts, probabilities > 1)
- Variance assumptions are violated (variance often depends on mean)
- Inference is invalid (confidence intervals include impossible values)
- Relationships are misspecified (effects are often multiplicative, not additive)

Generalized Linear Models (GLMs) elegantly solve these problems by extending the linear regression framework in two key ways.

## Poisson regression: Unbounded count data

This model is inspired by [a project by Ian Osvald](http://ianozsvald.com/2016/05/07/statistically-solving-sneezes-and-sniffles-a-work-in-progress-report-at-pydatalondon-2016/), which is concerned with understanding the various effects of external environmental factors upon the allergic sneezing of a test subject.

We're going to work with simpler data than the original study, which will allow you to clearly see the modeling stakes.

In [ ]:
sneezes = pl.read_csv("../data/poisson_sneeze.csv")
sneezes.head()

+ The subject sneezes N times per day, recorded as `nsneeze (int)`. The data are aggregated per day, to yield a total count of sneezes on that day.
+ The subject may or may not drink alcohol during that day, recorded as `alcohol (boolean)`
+ The subject may or may not take an antihistamine medication during that day, recorded as `meds (boolean)`

We assume that sneezing occurs at some baseline rate, which increases if an antihistamine is not taken, and further increases if alcohol is consumed.

### Visualize the data and set up the model

In [ ]:
fig = make_subplots(rows=2, cols=2, 
                   subplot_titles=["meds=0, alcohol=0", "meds=1, alcohol=0",
                                  "meds=0, alcohol=1", "meds=1, alcohol=1"])

for i, alc in enumerate([0, 1]):
    for j, med in enumerate([0, 1]):
        filtered_data = sneezes.filter((pl.col('alcohol') == alc) & (pl.col('meds') == med))
        counts = filtered_data.get_column('nsneeze').value_counts().sort('nsneeze')
        
        fig.add_trace(
            go.Bar(x=counts.get_column('nsneeze'), y=counts.get_column('count'), marker_color='blue'),
            row=i+1, col=j+1
        )

fig.update_layout(
    height=600, 
    width=800,
    showlegend=False,
    title_text="Distribution of Sneezes by Medication and Alcohol"
).update_xaxes(
    title_text="Number of Sneezes"
).update_yaxes(title_text="Count")


The usual way of thinking about data coming from a Poisson distribution is as number of occurences of an event in a *given timeframe*. Here *number of sneezes per day*. The intensity parameter $\lambda$ specifies how many occurences we expect.

A nice property of the Poisson is that it's defined with only one parameter $\lambda$, which describes both the mean and variance of the Poisson, can be interpreted as the rate of events per unit -- here, if we inferred $\lambda = 2.8$, that would mean the subject is thought to sneeze about 2.8 times per day (implying in addition a $2.8$ variance).

In statistical terms, that means our likelihood is 

$$ Y_{\text{sneeze}} \sim \mathrm{Poisson}(\lambda)$$

> The Poisson probability mass function is:
>
> $$ P(Y=k|\lambda) = \frac{\lambda^k e^{-\lambda}}{k!} $$
>
> where $k$ is the number of occurrences (sneezes in our case) and $\lambda$ is the rate parameter.
>


Now, we need a prior on $\lambda$ ...

This is where the regression component comes in: remember that we want to infer the effect of meds and alcohol on the number of sneezes. So, we can use the usual linear regression formula we are familiar with: 

$$\lambda = \alpha + \beta_{\text{meds}} * \text{meds} + \beta_{\text{alcohol}} * \text{alcohol}$$

We will specify **weakly-informative priors** on the model latent variables.

$$
\begin{aligned}
\alpha &\sim \mathrm{Normal}(0, 5)\\
\beta_{\text{meds}}, \beta_{\text{alcohol}} &\sim \mathrm{Normal}(0, 1)\\
\lambda &= \alpha + \beta_{\text{meds}} * \text{meds} + \beta_{\text{alcohol}} * \text{alcohol}\\
Y_{\text{sneeze}} &\sim \mathrm{Poisson}(\lambda)\\
\end{aligned}
$$

In [ ]:
COORDS = {
    "regressor": ["meds", "alcohol"], 
    "obs_idx": range(len(sneezes))
}

M, A, S = sneezes.select(["meds", "alcohol", "nsneeze"]).to_numpy().T

with pm.Model(coords=COORDS) as m_sneeze:
    # weakly informative Normal Priors
    a = pm.Normal("intercept", mu=0, sigma=5)
    b = pm.Normal("slopes", mu=0, sigma=1, dims="regressor")

    # define linear model
    mu = pm.Deterministic("mu", a + b[0] * M + b[1] * A, dims="obs_idx")

    ## Define Poisson likelihood
    y = pm.Poisson("y", mu=mu, observed=S, dims="obs_idx")

### Model Fitting

Most GLMs will be fit using the NUTS step method.

In [ ]:
with m_sneeze:
    trace_sneeze = pm.sample(random_seed=RANDOM_SEED)

In [ ]:
az.summary(trace_sneeze, var_names=["intercept", "slopes"])

While the model sampled to completion, close inspection of the model reveals an issue that could have caused the sampler to fail. Can you spot it? In previous versions of PyMC, this model would have failed to sample.

Propose a more robust version below.

In [ ]:
with pm.Model(coords=COORDS) as m_sneeze:
    # weakly informative Normal Priors
    a = pm.Normal("intercept", mu=0, sigma=5)
    b = pm.Normal("slopes", mu=0, sigma=1, dims="regressor")

    # define linear model
    mu = pm.math.exp(a + b[0] * M + b[1] * A)

    ## Define Poisson likelihood
    y = pm.Poisson("y", mu=mu, observed=S, dims="obs_idx")

    trace_sneeze = pm.sample(random_seed=RANDOM_SEED)

Before looking at the results, let's take a step back: the trick we used with the exponential transformation is an example of a generalized linear model. The exponential here is called **a link function**, and it's used to map the **output of our linear model** (a priori allowed to live in $(-\infty, \infty)$,

### Link functions

In Generalized Linear Models (GLMs), link functions play a crucial role in connecting the linear predictor to the response variable. The concept of link functions arises from the need to model the relationship between the mean of the response variable and the linear predictor, which can take any real value.

The link function transforms the linear predictor to ensure that the predicted values of the response variable are within a valid range and satisfy the distributional assumptions of the response variable. It maps the linear predictor to the space of the relevant parameter of the response distribution.

Different types of GLMs use different link functions based on the nature of the response variable and the desired distribution. Some commonly used link functions include:

1. Identity Link: This link function is used for continuous response variables and maintains a linear relationship between the linear predictor and the mean of the response variable.

2. Logit Link: This link function is used for binary response variables and maps the linear predictor to the probability of success in a logistic regression model. It ensures that the predicted probabilities are between 0 and 1.

3. Log Link: This link function is used for count data and maps the linear predictor to the mean of a Poisson distribution. It ensures that the predicted mean is positive.

4. Inverse Link: This link function is used for modeling the mean of a Gamma distribution and maps the linear predictor to the inverse of the mean.

The choice of the link function depends on the nature of the response variable and the assumptions of the distribution. The link function allows for flexible modeling of the relationship between the linear predictor and the response variable, enabling the estimation of regression coefficients and making predictions within the appropriate range of the response distribution.

### Checking our inferences

Now is the time to check if our model's results are credible, via posterior predictive checks, to which we were introduced in the previous section.

In [ ]:
with m_sneeze:

    # Get posterior predictive samples, and add them to the InferenceData object
    trace_sneeze.extend(pm.sample_posterior_predictive(trace_sneeze))

Let's plot the posterior predictive checks for all the groups.

In [ ]:
def plot_sneeze_predictions(idata):
    fig, axs = plt.subplots(2, 2, figsize=(12, 8))

    az.plot_ppc(
        idata,
        ax=axs[0, 0],
        coords={
            "obs_idx": np.where(np.logical_and(sneezes.get_column("alcohol").to_numpy() == 0, sneezes.get_column("meds").to_numpy() == 0))
        },
    )
    az.plot_ppc(
        idata,
        ax=axs[0, 1],
        coords={
            "obs_idx": np.where(np.logical_and(sneezes.get_column("alcohol").to_numpy() == 0, sneezes.get_column("meds").to_numpy() == 1))
        },
    )
    az.plot_ppc(
        idata,
        ax=axs[1, 0],
        coords={
            "obs_idx": np.where(np.logical_and(sneezes.get_column("alcohol").to_numpy() == 1, sneezes.get_column("meds").to_numpy() == 0))
        },
    )
    az.plot_ppc(
        idata,
        ax=axs[1, 1],
        coords={
            "obs_idx": np.where(np.logical_and(sneezes.get_column("alcohol").to_numpy() == 1, sneezes.get_column("meds").to_numpy() == 1))
        },
    )
    axs[0, 0].set_title("No alcohol : No meds")
    axs[0, 1].set_title("No alcohol : Meds")
    axs[1, 0].set_title("Alcohol : No meds")
    axs[1, 1].set_title("Alcohol : Meds")
    return fig, axs

In [ ]:
plot_sneeze_predictions(trace_sneeze);

While the model is in the ballpark of the actual data, it is not perfect. The model is underestimating the variance in the data. This is a common issue with Poisson regression, as the variance is constrained to be equal to the mean. When the data's mean and variance are not similar, a Poisson regression will underestimate the variance compared to the true variance observed in the data.

This behavior is quite common with Poisson regression: it often underestimates the variation in the data, simply because real data are more dispersed than our regression expects -- in these cases, data are said to be "overdispersed".

This phenomenon is particularly acute with the Poisson, because as we have seen its variance is mathematically constrained to be equal to its mean. So, when the data's mean and variance are not similar, a Poisson regression will get the variance estimate wrong when compared to the true variance observed in the data.

To convince ourselves, let's compare our data's mean and variance:

In [ ]:
sneezes.group_by(["meds", "alcohol"]).agg([
    pl.col("nsneeze").mean().alias("mean"),
    pl.col("nsneeze").var().alias("var")
]).sort(["meds", "alcohol"])

Notice that for each combination of `alcohol` and `meds`, the variance of `nsneeze` is higher than the mean!

## Gamma-Poisson Model

Gamma-Poisson (aka [Negative binomial](https://en.wikipedia.org/wiki/Negative_binomial_distribution)) regression is used to model overdispersion in count data. The Gamma-Poisson distribution can be thought of as a Poisson distribution whose rate parameter is gamma distributed, so that rate parameter can be adjusted to account for the increased variance. If you want more details about these models (a.k.a. "continuous mixture models"), I refer you to chapter 12 of [Richard McElreath's excellent _Statistical Rethinking_](https://nbviewer.jupyter.org/github/pymc-devs/resources/blob/master/Rethinking_2/Chp_12.ipynb).

In addition to the Poisson rate, $\lambda$, Gamma-Poisson distributions are parametrized an additional overdispersion parameter, $\alpha$, which controls the shape of the Gamma distribution. 

_The Gamma-Poisson distribution_

We start with a random variable $Y$ that follows a Poisson distribution with rate $\lambda$. Turns out $\lambda$ is also random, and it follows a gamma distribution with parameters $\mu$ and $\alpha$

$$
\begin{aligned}
Y &\sim \text{Poisson}(\lambda) \\
\lambda &\sim \text{Gamma}\left(\mu, \alpha\right)
\end{aligned}
$$


We can marginalize over $\lambda$


$$
\begin{aligned}
p(y \mid \mu, \alpha) &= \int_0^{\infty}{p(y \mid \lambda) p(\lambda \mid \mu, \alpha) d\lambda} \\
&= \binom{y + \alpha - 1}{y}{\left(\frac{\alpha}{\mu + \alpha}\right)}^\alpha {\left(\frac{\mu}{\mu + \alpha}\right)}^y
\end{aligned}
$$


The above describes the probability mass function of a Gamma-Poisson distribution, then we can say 

$$
Y \sim \text{GammaPoisson}(\mu, \alpha)
$$


### Why is it useful?

Well, it relieves us from the previous constraint of our Poisson Distribution, to fix the **mean** to the **variance**.

<br> </br>

<center>
  <img src="images/poisson_gamma_poisson_drake.png" style="width:500px"; />
</center>

The common name for this type of model is [negative binomial](https://en.wikipedia.org/wiki/Negative_binomial_distribution) regression.  The negative binomial distribution has two parameters: the mean $\mu$ and the overdispersion parameter $\alpha$. The mean $\mu$ is the rate parameter of the Poisson distribution, and the overdispersion parameter $\alpha$ controls the variance of the distribution.

We'll use the following model...


$$
\begin{aligned}
\beta_{\text{intercept}} &\sim \mathrm{Normal}(0, 5)\\
\beta_{\text{alcohol}} &\sim \mathrm{Normal}(0, 1)\\
\beta_{\text{meds}} &\sim \mathrm{Normal}(0, 1) \\
\alpha &\sim \mathrm{Exponential}(1) \\
\mu_i &= \exp(\beta_{\text{intercept}} + \beta_{\text{meds}} \text{meds}_i + \beta_{\text{alcohol}} \text{alcohol}_i) \\
Y \mid \mu_i, \alpha &\sim \mathrm{NegativeBinomial}(\mu_i, \alpha) \\
\end{aligned}
$$

Let's use this likelihood in PyMC:

In [ ]:
with pm.Model(coords=COORDS) as m_sneeze_gp:
    # weakly informative priors
    a = pm.Normal("intercept", mu=0, sigma=5)
    b = pm.Normal("slopes", mu=0, sigma=1, dims="regressor")
    alpha = pm.Exponential("alpha", 1.0)

    # define linear model
    mu = pm.math.exp(a + b[0] * M + b[1] * A)

    ## likelihood
    y = pm.NegativeBinomial("y", mu=mu, alpha=alpha, observed=S, dims="obs_idx")

    trace_sneeze_gp = pm.sample(random_seed=RANDOM_SEED)

In [ ]:
az.plot_trace(trace_sneeze_gp);

Sampling went well; let's quickly check the fit.

In [ ]:
with m_sneeze_gp:
    trace_sneeze_gp.extend(pm.sample_posterior_predictive(trace_sneeze_gp))

plot_sneeze_predictions(trace_sneeze_gp);

### Exercise: Interaction effect

The predictions look much better than before, **but** we can see there is bias relative to the observations in some groups. For example, in the plot for no alcohol with medication the model is biased to predict fewer sneezes than we actually observe in the medication condition, and the opposite phenomenon in the no medication condition. 

This suggests that we are missing some sort of **interaction effect** between medication and alcohol consumption. The thing is that our model is only able to account for the mean sneezes across both conditions.

Try your hand at adding an interaction term to the model:

In [ ]:
COORDS = {
		"regressor": ["meds", "alcohol"], 
		"obs_idx": range(len(sneezes))
}
with pm.Model(coords=COORDS) as m_sneeze_inter:

    # weakly informative priors
    a = pm.Normal("intercept", mu=0, sigma=5)
    b = pm.Normal("slopes", mu=0, sigma=1, dims="regressor")
    alpha = pm.Exponential("alpha", 1.0)

    # define linear model
    mu = pm.math.exp(a + b[0] * M + b[1] * A)

    ## likelihood
    y = pm.NegativeBinomial("y", mu=mu, alpha=alpha, observed=S, dims="obs_idx")

    trace_sneeze_inter = pm.sample(random_seed=RANDOM_SEED)

We see that the slope for the interaction is reliably negative, meaning that taking meds when drinking alcohol will still tame the effects of the latter on sneezing and thus decrease the number of sneezes compared to not taking meds.

In [ ]:
with m_sneeze_inter:
    trace_sneeze_inter.extend(pm.sample_posterior_predictive(trace_sneeze_inter))

plot_sneeze_predictions(trace_sneeze_inter);

We can see that the interaction term appears to have removed the biases, and the predictions look much better across the board.

## Generalized Linear Models 

Poisson and negative binomial regressions are particular types of **Generalized Linear Model**. 

### Linear Models

We assume the conditional distribution of the response variable is a normal distribution. We model the mean of that normal distribution with a linear combination of the predictors. Mathematically, we have

$$
\begin{aligned}
\pmb{\beta} &\sim \mathcal{P}_{\pmb{\beta}} \\
\sigma &\sim \mathcal{P}_\sigma \\
\mu_i &= \beta_0 + \beta_1 X_{1i} + \cdots + \beta_p X_{pi} \\
Y_i \mid \mu_i, \sigma &\sim \text{Normal}(\mu_i, \sigma)
\end{aligned}
$$

where $\mathcal{P}_{\pmb{\beta}}$ is the joint prior for the regression coefficients and $\mathcal{P}_\sigma$ is the prior on the residual standard deviation.

### Generalized Linear Models

In Generalized Linear Models, we are not restricted to normal likelihoods and we model a function of the mean with a linear combination of the predictors.

$$
\begin{aligned}
\pmb{\beta} &\sim \mathcal{P}_{\pmb{\beta}} \\
\pmb{\theta} &\sim \mathcal{P}_{\pmb{\theta}} \\
g(\mu_i) &= \eta_i = \beta_0 + \beta_1 X_{1i} + \cdots + \beta_p X_{pi} \\
Y_i \mid \mu_i, \pmb{\theta} &\sim \mathcal{D}(\mu_i, \pmb{\theta})
\end{aligned}
$$

Which consists of:

* $\eta_i = \beta_0 + \beta_1 X_{1i} + \cdots + \beta_p X_{pi}$ is the **linear predictor**
* $g$ is the **link function**
    * In the Poisson regression model $g$ is the $\log$ function.
    * This point raises a lot of questions, as we directly work with the inverse link function $g^{-1}$ ($\exp$ in the previous case)
    * $g: \Omega \to \mathbb{R}$
    * $g^{-1}: \mathbb{R} \to \Omega$
    * $\Omega$ is the space of the mean parameter
* $\mathcal{D}$ is the **sampling distribution**
    * The conditional distribution of the response variable $Y$
    * Is not necessarily normal

Linear models are specific case of generalized linear models where $\mathcal{D} \equiv \mathcal{N}$ and $g = I$ (*i.e.* the identity function).

## Hierarchical Models


Hierarchical or multilevel modeling is another generalization of regression modeling.

*Multilevel models* are regression models in which the constituent model parameters are given **probability models**. This implies that model parameters are allowed to **vary by group**.

Observational units are often naturally **clustered**. Clustering induces dependence between observations, despite random sampling of clusters and random sampling within clusters.

A *hierarchical model* is a particular multilevel model where parameters are nested within one another.

Some multilevel structures are not hierarchical. 

* e.g. "country" and "year" are not nested, but may represent separate, but overlapping, clusters of parameters

We will motivate this topic using an environmental epidemiology example.

### Example: Radon contamination (Gelman and Hill 2006)

Radon is a radioactive gas that enters homes through contact points with the ground. It is a carcinogen that is the primary cause of lung cancer in non-smokers. Radon levels vary greatly from household to household.

![radon](https://www.cgenarchive.org/uploads/2/5/2/6/25269392/7758459_orig.jpg)

The EPA did a study of radon levels in 80,000 houses. There are two important predictors:

* measurement in basement or first floor (radon higher in basements)
* county uranium level (positive correlation with radon levels)

We will focus on modeling radon levels in Minnesota.

The hierarchy in this example is households within county.

### Data organization

First, we import the data from a local file, and extract Minnesota's data.

The original data exists as several independent datasets, which we will import, merge, and process here. First is the data on measurements from individual homes from across the United States. We will extract just the subset from Minnesota.

In [ ]:
srrs2 = pl.read_csv('../data/srrs2.dat')

srrs2 = srrs2.rename({col: col.strip() for col in srrs2.columns})
srrs_mn = srrs2.filter(pl.col("state") == "MN")
srrs_mn.shape

Next, obtain the county-level predictor, uranium, by combining two variables.

In [ ]:
cty = pl.read_csv('../data/cty.dat')

srrs_mn = srrs_mn.with_columns(
    (pl.col("stfips").str.strip_chars().cast(pl.Int64) * 1000 + pl.col("cntyfips").str.strip_chars().cast(pl.Int64)).alias("fips")
)
cty_mn = cty.filter(pl.col("st") == "MN").with_columns(
    (1000 * pl.col("stfips") + pl.col("ctfips")).alias("fips")
)

Use the Polars `join` method to combine home- and county-level information in a single DataFrame.

In [ ]:
srrs_mn = srrs_mn.join(cty_mn.select(["fips", "Uppm"]), on="fips")
srrs_mn = srrs_mn.unique(subset=["idnum"], maintain_order=True)

n = len(srrs_mn)

Let's encode the county names and make local copies of the variables we will use.
We also need a lookup table (`dict`) for each unique county, for indexing.

In [ ]:
srrs_mn = srrs_mn.with_columns(
    pl.col("county").map_elements(str.strip, return_dtype=pl.Utf8).alias("county")
)

unique_counties = srrs_mn.select("county").unique(maintain_order=True).to_series().to_list()
mn_counties = np.array(unique_counties)
county_dict = {county: i for i, county in enumerate(unique_counties)}


county_uranium_df = (srrs_mn.group_by("county", maintain_order=True).agg(pl.col("Uppm").first()))

county_to_uranium = {row[0]: row[1] for row in county_uranium_df.iter_rows()}

ordered_uranium = [county_to_uranium[county] for county in unique_counties]
u = np.log(np.array(ordered_uranium))

srrs_mn = srrs_mn.with_columns(
    pl.col("county").replace_strict(county_dict, default=None).alias("county_code")
)

srrs_mn = srrs_mn.with_columns(
    pl.col("activity").str.strip_chars().cast(pl.Float64).alias("activity")
)

county = srrs_mn.select("county_code").to_numpy().flatten()
radon = srrs_mn.select("activity").to_numpy().flatten()
log_radon = np.log(radon + 0.1)
srrs_mn = srrs_mn.with_columns(
pl.lit(log_radon).alias("log_radon")
)
floor_measure = srrs_mn.select("floor").to_numpy().flatten()

Distribution of radon levels in MN (log scale):

In [ ]:
px.histogram(
    srrs_mn.to_pandas(),  # polars to pandas for plotly compatibility
    x="log_radon",
    nbins=50,
    labels={"log_radon": "log(radon)"},
    title="Distribution of log(radon) levels in MN"
).update_layout(
    xaxis_title="log(radon)",
    yaxis_title="frequency"
)

## Conventional approaches

The two conventional alternatives to modeling radon exposure represent the two extremes of the bias-variance tradeoff:

***Complete pooling***: 

Treat all counties the same, and estimate a single radon level.

$$y_i = \alpha + \beta x_i + \epsilon_i$$

***No pooling***:

Model radon in each county independently.

$$y_i = \alpha_{j[i]} + \beta x_i + \epsilon_i$$

where $j = 1,\ldots,85$

The errors $\epsilon_i$ may represent measurement error, temporal within-house variation, or variation among houses.

Here are the point estimates of the slope and intercept for the complete pooling model:

In [ ]:
with pm.Model() as pooled_model:
    floor_ind = pm.Data("floor_ind", floor_measure, dims="obs_id")

    alpha = pm.Normal("alpha", 0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10)
    sigma = pm.Exponential("sigma", 5)

    theta = alpha + beta * floor_ind

    y = pm.Normal("y", theta, sigma=sigma, observed=log_radon, dims="obs_id")

In [ ]:
pm.model_to_graphviz(pooled_model)

You may be wondering why we are using the `pm.Data` container above even though the variable `floor_ind` is not an observed variable nor a parameter of the model. As you'll see, this will make our lives much easier when we'll plot and diagnose our model.ArviZ will thus include `floor_ind` as a variable in the `constant_data` group of the resulting {ref}`InferenceData <xarray_for_arviz>` object. Moreover, including `floor_ind` in the `InferenceData` object makes sharing and reproducing analysis much easier, all the data needed to analyze or rerun the model is stored there.

Before running the model let's do some **prior predictive checks**. 

Indeed, having sensible priors is not only a way to incorporate scientific knowledge into the model, it can also help and make the MCMC machinery faster -- here we are dealing with a simple linear regression, so no link function comes and distorts the outcome space; but one day this will happen to you and you'll need to think hard about your priors to help your MCMC sampler. So, better to train ourselves when it's quite easy than having to learn when it's very hard. 

In [ ]:
with pooled_model:
    prior_checks = pm.sample_prior_predictive(random_seed=RANDOM_SEED)

ArviZ `InferenceData` uses `xarray.Dataset`s under the hood, which give access to several common plotting functions with `.plot`. In this case, we want scatter plot of the mean log radon level (which is stored in variable `a`) for each of the two levels we are considering. If our desired plot is supported by xarray plotting capabilities, we can take advantage of xarray to automatically generate both plot and labels for us. Notice how everything is directly plotted and annotated, the only change we need to do is renaming the y axis label from `a` to `Mean log radon level`.

In [ ]:
prior = prior_checks.prior.squeeze(drop=True)

xr.concat((prior["alpha"], prior["alpha"] + prior["beta"]), dim="location").rename(
    "log_radon"
).assign_coords(location=["basement", "floor"]).plot.scatter(
    x="location", y="log_radon", edgecolors="none"
);

I'm no radon expert, but before seeing the data, these priors seem to allow for quite a wide range of the mean log radon level, both as measured either in a basement or on a floor. But don't worry, we can always change these priors if sampling gives us hints that they might not be appropriate -- after all, priors are assumptions, not oaths; and as with most assumptions, they can be tested.

However, we can already think of an improvement: Remember that we stated radon levels tend to be higher in basements, so we could incorporate this prior scientific knowledge into our model by forcing the floor effect (`beta`) to be negative. For now, we will leave the model as is, and trust that the information in the data will be sufficient.

Speaking of sampling, let's fire up the Bayesian machinery!

In [ ]:
with pooled_model:
    pooled_trace = pm.sample(random_seed=RANDOM_SEED)

No divergences and a sampling that only took seconds! Here the chains look very good (good R hat, good effective sample size, small sd). The model also estimated a negative floor effect, as we expected.

In [ ]:
az.summary(pooled_trace, round_to=2)

Let's plot the expected radon levels in basements (`alpha`) and on floors (`alpha + beta`) in relation to the data used to fit the model:

In [ ]:
post_mean = pooled_trace.posterior.mean(dim=("chain", "draw"))
xvals = np.linspace(-0.2, 1.2, 100)

px.scatter(
    x=srrs_mn["floor"].to_numpy(),
    y=np.log(srrs_mn["activity"].to_numpy() + 0.1),
    labels={"x": "floor", "y": "log(activity + 0.1)"}
).add_scatter(
    x=xvals,
    y=post_mean["beta"].item() * xvals + post_mean["alpha"].item(),
    mode="lines",
    line=dict(dash="dash", color="red"),
    name="Posterior Mean"
)

This looks reasonable, though notice that there is a great deal of residual variability in the data. 

Let's now turn our attention to the unpooled model, and see how it fares in comparison.

In [ ]:
coords = {"county": mn_counties}

with pm.Model(coords=coords) as unpooled_model:
    floor_ind = pm.Data("floor_ind", floor_measure, dims="obs_id")

    alpha = pm.Normal("alpha", 0, sigma=10, dims="county")
    beta = pm.Normal("beta", 0, sigma=10)
    sigma = pm.Exponential("sigma", 1)

    theta = alpha[county] + beta * floor_ind

    y = pm.Normal("y", theta, sigma=sigma, observed=log_radon, dims="obs_id")

In [ ]:
pm.model_to_graphviz(unpooled_model)

In [ ]:
with unpooled_model:
    unpooled_trace = pm.sample(random_seed=RANDOM_SEED)

The sampling was clean here too; Let's look at the expected values for both basement (dimension 0) and floor (dimension 1) in each county:

In [ ]:
ax = az.plot_forest(
    unpooled_trace,
    var_names=["alpha"],
    r_hat=True,
    combined=True,
    figsize=(6, 18),
    labeller=az.labels.NoVarLabeller(),
)
ax[0].set_ylabel("alpha");

To identify counties with high radon levels, we can plot the ordered mean estimates, as well as their 94% HPD:

In [ ]:
unpooled_means = unpooled_trace.posterior.mean(dim=("chain", "draw"))
unpooled_hdi = az.hdi(unpooled_trace)

unpooled_means_iter = unpooled_means.sortby("alpha")
unpooled_hdi_iter = unpooled_hdi.sortby(unpooled_means_iter.alpha)

_, ax = plt.subplots(figsize=(10,6))
xticks = np.arange(0, 86, 6)
unpooled_means_iter.plot.scatter(x="county", y="alpha", ax=ax, alpha=0.8)
ax.vlines(
    np.arange(mn_counties.shape[0]),
    unpooled_hdi_iter.alpha.sel(hdi="lower"),
    unpooled_hdi_iter.alpha.sel(hdi="higher"),
    color="orange",
    alpha=0.6,
)
ax.set(ylabel="Radon estimate", ylim=(-2, 4.5))
ax.set_xticks(xticks)
ax.set_xticklabels(unpooled_means_iter.county.values[xticks])
ax.tick_params(rotation=45);

Now that we have fit both conventional (*i.e.* non-hierarchcial) models, let's see how their inferences differ. Here are visual comparisons between the pooled and unpooled estimates for a subset of counties representing a range of sample sizes.

In [ ]:
sample_counties = (
    "LAC QUI PARLE",
    "AITKIN",
    "KOOCHICHING",
    "DOUGLAS",
    "CLAY",
    "STEARNS",
    "RAMSEY",
    "ST LOUIS",
)

fig, axes = plt.subplots(2, 4, figsize=(12, 6), sharey=True, sharex=True)
axes = axes.ravel()
m = unpooled_means["beta"]
for i, c in enumerate(sample_counties):
    y = srrs_mn.filter(pl.col("county") == c)["log_radon"].to_numpy()
    x = srrs_mn.filter(pl.col("county") == c)["floor"].to_numpy()
    axes[i].scatter(x + np.random.randn(len(x)) * 0.01, y, alpha=0.4)

    # No pooling model
    b = unpooled_means["alpha"].sel(county=c)

    # Plot both models and data
    xvals = xr.DataArray(np.linspace(0, 1))
    axes[i].plot(xvals, m * xvals + b)
    axes[i].plot(xvals, post_mean["beta"] * xvals + post_mean["alpha"], "r--")
    axes[i].set_xticks([0, 1])
    axes[i].set_xticklabels(["basement", "floor"])
    axes[i].set_ylim(-1, 3)
    axes[i].set_title(c)
    if not i % 2:
        axes[i].set_ylabel("log radon level")

Neither of these models are satisfactory:

* If we are trying to identify high-radon counties, pooling is useless -- because, by definition, the pooled model estimates radon at the state-level. In other words, pooling leads to maximal *underfitting*: the variation across counties is not taken into account and only the overall population is estimated.
* We do not trust extreme unpooled estimates produced by models using few observations. This leads to maximal *overfitting*: only the within-county variations are taken into account and the overall population (i.e the state-level, which tells us about similarities across counties) is not estimated. 

This issue is acute for small sample sizes, as seen above: in counties where we have few floor measurements, if radon levels are higher for those data points than for basement ones (Aitkin, Koochiching, Ramsey), the model will estimate that radon levels are higher in floors than basements for these counties. But we shouldn't trust this conclusion, because both scientific knowledge and the situation in other counties tell us that it is usually the reverse (basement radon > floor radon). So unless we have a lot of observations telling us otherwise for a given county, we should be skeptical and shrink our county-estimates to the state-estimates -- in other words, we should balance between cluster-level and population-level information, and the amount of shrinkage will depend on how extreme and how numerous the data in each cluster are. 

Here is where hierarchical models come into play.

## Multilevel and hierarchical models

When we pool our data, we imply that they are sampled from the same model. This ignores any variation among sampling units (other than sampling variance) -- we assume that counties are all the same:

![pooled](images/pooled_model.png)

When we analyze data unpooled, we imply that they are sampled independently from separate models. At the opposite extreme from the pooled case, this approach claims that differences between sampling units are too large to combine them -- we assume that counties have no similarity whatsoever:

![unpooled](images/unpooled_model.png)

In a hierarchical model, parameters are viewed as a sample from a population distribution of parameters. Thus, we view them as being neither entirely different or exactly the same. This is ***partial pooling***:

![hierarchical](images/partial_pooled_model.png)

We can use PyMC to easily specify multilevel models, and fit them using Markov chain Monte Carlo.

## Partial pooling model

The simplest partial pooling model for the household radon dataset is one which simply estimates radon levels, without any predictors at any level. A partial pooling model represents a compromise between the pooled and unpooled extremes, essentially a weighted average (based on sample size) of the unpooled county estimates and the pooled estimates.

$$\hat{\alpha} \approx \frac{(n_j/\sigma_y^2)\bar{y}_j + (1/\sigma_{\alpha}^2)\bar{y}}{(n_j/\sigma_y^2) + (1/\sigma_{\alpha}^2)}$$

Estimates for counties with smaller sample sizes will shrink towards the state-wide average, while those for counties with larger sample sizes will be closer to the unpooled county estimates.

Let's start with a very simple partial pooling model, which ignores the effect of floor vs. basement measurement.

In [ ]:
with pm.Model(coords=coords) as partial_pooling:
    county_idx = pm.Data("county_idx", county, dims="obs_id")

    # Priors
    mu_a = pm.Normal("mu_a", mu=0.0, sigma=10)
    sigma_a = pm.Exponential("sigma_a", 1)

    # Random intercepts
    alpha = pm.Normal("alpha", mu=mu_a, sigma=sigma_a, dims="county")

    # Model error
    sigma_y = pm.Exponential("sigma_y", 1)

    # Expected value
    y_hat = alpha[county_idx]

    # Data likelihood
    y_like = pm.Normal("y_like", mu=y_hat, sigma=sigma_y, observed=log_radon, dims="obs_id")

In [ ]:
pm.model_to_graphviz(partial_pooling)

In [ ]:
with partial_pooling:
    partial_pooling_trace = pm.sample(tune=2000, random_seed=RANDOM_SEED)

In [ ]:
N_county = srrs_mn.group_by("county").agg(pl.count("idnum")).sort("county")["idnum"].to_numpy()

fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharex=True, sharey=True)
for ax, trace, level in zip(
    axes,
    (unpooled_trace, partial_pooling_trace),
    ("no pooling", "partial pooling"),
):
    # add variable with x values to xarray dataset
    trace.posterior = trace.posterior.assign_coords({"N_county": ("county", N_county)})
    # plot means
    trace.posterior.mean(dim=("chain", "draw")).plot.scatter(
        x="N_county", y="alpha", ax=ax, alpha=0.9
    )
    ax.hlines(
        partial_pooling_trace.posterior.alpha.mean(),
        0.9,
        max(N_county) + 1,
        alpha=0.4,
        ls="--",
        label="Est. population mean",
    )

    # plot hdi
    hdi = az.hdi(trace).alpha
    ax.vlines(N_county, hdi.sel(hdi="lower"), hdi.sel(hdi="higher"), color="orange", alpha=0.5)

    ax.set(
        title=f"{level.title()} Estimates",
        xlabel="Nbr obs in county (log scale)",
        xscale="log",
        ylabel="Log radon",
    )
    ax.legend(fontsize=10)

Notice the difference between the unpooled and partially-pooled estimates, particularly at smaller sample sizes: As expected, the former are both more extreme and more imprecise. Indeed, in the partially-pooled model, estimates in small-sample-size counties are informed by the population parameters -- hence more precise estimates. Moreover, the smaller the sample size, the more regression towards the overall mean (the dashed gray line) -- hence less extreme estimates. In other words, the model is skeptical of extreme deviations from the population mean in counties where data is sparse. This is known as **shrinkage**.

Now let's go back and integrate the `floor` predictor, but allowing the intercept to vary by county.

## Varying intercept model

This model allows intercepts to vary across county, according to a random effect.

$$y_i = \alpha_{j[i]} + \beta x_{i} + \epsilon_i$$

where

$$\epsilon_i \sim N(0, \sigma_y^2)$$

and the intercept random effect:

$$\alpha_{j[i]} \sim N(\mu_{\alpha}, \sigma_{\alpha}^2)$$

As with the the “no-pooling” model, we set a separate intercept for each county, but rather than fitting separate least squares regression models for each county, multilevel modeling **shares strength** among counties, allowing for more reasonable inference in counties with little data.

In [ ]:
with pm.Model(coords=coords) as varying_intercept:
    floor_idx = pm.Data("floor_idx", floor_measure, dims="obs_id")
    county_idx = pm.Data("county_idx", county, dims="obs_id")

    # Priors
    mu_a = pm.Normal("mu_a", mu=0.0, sigma=10.0)
    sigma_a = pm.Exponential("sigma_a", 1)

    # Random intercepts
    alpha = pm.Normal("alpha", mu=mu_a, sigma=sigma_a, dims="county")
    # Common slope
    beta = pm.Normal("beta", mu=0.0, sigma=10.0)

    # Model error
    sd_y = pm.Exponential("sd_y", 1)

    # Expected value
    y_hat = alpha[county_idx] + beta * floor_idx

    # Data likelihood
    y_like = pm.Normal("y_like", mu=y_hat, sigma=sd_y, observed=log_radon, dims="obs_id")

In [ ]:
pm.model_to_graphviz(varying_intercept)

In [ ]:
with varying_intercept:
    varying_intercept_trace = pm.sample(tune=2000, random_seed=RANDOM_SEED)

In [ ]:
ax = az.plot_forest(
    varying_intercept_trace,
    var_names=["alpha"],
    figsize=(6, 18),
    combined=True,
    r_hat=True,
    labeller=az.labels.NoVarLabeller(),
)
ax[0].set_ylabel("alpha");

In [ ]:
az.plot_posterior(varying_intercept_trace, var_names=["sigma_a", "beta"]);

The estimate for the `floor` coefficient is approximately -0.66, which can be interpreted as houses without basements having about half ($\exp(-0.66) = 0.52$) the radon levels of those with basements, after accounting for county.

In [ ]:
az.summary(varying_intercept_trace, var_names=["beta"])

In [ ]:
xvals = xr.DataArray([0, 1], dims="Level", coords={"Level": ["Basement", "Floor"]})
post = varying_intercept_trace.posterior  # alias for readability
theta = (
    (post.alpha + post.beta * xvals).mean(dim=("chain", "draw")).to_dataset(name="Mean log radon")
)

_, ax = plt.subplots()
theta.plot.scatter(x="Level", y="Mean log radon", alpha=0.2, color="k", ax=ax)  # scatter
ax.plot(xvals, theta["Mean log radon"].T, "k-", alpha=0.2)
# add lines too
ax.set_title("MEAN LOG RADON BY COUNTY");

It is easy to show that the partial pooling model provides more objectively reasonable estimates than either the pooled or unpooled models, at least for counties with small sample sizes.

In [ ]:
sample_counties = (
    "LAC QUI PARLE",
    "AITKIN",
    "KOOCHICHING",
    "DOUGLAS",
    "CLAY",
    "STEARNS",
    "RAMSEY",
    "ST LOUIS",
)

fig, axes = plt.subplots(2, 4, figsize=(12, 6), sharey=True, sharex=True)
axes = axes.ravel()
m = unpooled_means["beta"]
for i, c in enumerate(sample_counties):
    y = srrs_mn.filter(pl.col("county") == c)["log_radon"].to_numpy()
    x = srrs_mn.filter(pl.col("county") == c)["floor"].to_numpy()
    axes[i].scatter(x + np.random.randn(len(x)) * 0.01, y, alpha=0.4)

    # No pooling model
    b = unpooled_means["alpha"].sel(county=c)

    # Plot both models and data
    xvals = xr.DataArray(np.linspace(0, 1))
    axes[i].plot(xvals, m.values * xvals + b.values)
    axes[i].plot(xvals, post_mean["beta"] * xvals + post_mean["alpha"], "r--")

    varying_intercept_trace.posterior.sel(county=c).beta
    post = varying_intercept_trace.posterior.sel(county=c).mean(dim=("chain", "draw"))
    theta = post.alpha.values + post.beta.values * xvals
    axes[i].plot(xvals, theta, "k:")
    axes[i].set_xticks([0, 1])
    axes[i].set_xticklabels(["basement", "floor"])
    axes[i].set_ylim(-1, 3)
    axes[i].set_title(c)
    if not i % 2:
        axes[i].set_ylabel("log radon level")

### Exercise: Varying intercept and slope model

The most general model allows both the intercept and slope to vary by county:

$$y_i = \alpha_{j[i]} + \beta_{j[i]} x_{i} + \epsilon_i$$

Construct a model called `varying_intercept_slope` that implements this alternative model.

In [ ]:
with pm.Model(coords=coords) as varying_intercept_slope:
    
    # Write your model here

Plot the model DAG just to check your model is correct.

In [ ]:
pm.model_to_graphviz(varying_intercept_slope)

In [ ]:
with varying_intercept_slope:
    varying_intercept_slope_trace = pm.sample(tune=2000, random_seed=RANDOM_SEED)

Notice that the trace of this model includes divergences, which can be problematic depending on where and how frequently they occur. These can occur is some hierararchical models, and they can be avoided by using the **non-centered parametrization**.

## Non-centered Parameterization

The partial pooling models specified above uses a **centered** parameterization of the slope random effect. That is, the individual county effects are distributed around a county mean, with a spread controlled by the hierarchical standard deviation parameter. As the preceding plot reveals, this constraint serves to **shrink** county estimates toward the overall mean, to a degree proportional to the county sample size. This is exactly what we want, and the model appears to fit well--the Gelman-Rubin statistics are exactly 1.

But, on closer inspection, there are signs of trouble. Specifically, let's look at the trace of the random effects, and their corresponding standard deviation:

In [ ]:
# Extract posterior samples for chain 0 using polars
sigma_b_df = varying_intercept_slope_trace.posterior["sigma_b"].sel(chain=0).to_dataframe().reset_index()
beta_df = varying_intercept_slope_trace.posterior["beta"].sel(chain=0).to_dataframe().reset_index()

fig, axs = plt.subplots(nrows=2)
axs[0].plot(sigma_b_df["sigma_b"].to_numpy(), alpha=0.5)
axs[0].set(ylabel="sigma_b")
axs[1].plot(beta_df["beta"].to_numpy(), alpha=0.5)
axs[1].set(ylabel="beta");

Notice that when the chain reaches the lower end of the parameter space for $\sigma_b$, it appears to get "stuck" and the entire sampler, including the random slopes `beta`, mixes poorly. 

Jointly plotting the random effect variance and one of the individual random slopes demonstrates what is going on.

In [ ]:
x = varying_intercept_slope_trace.posterior["beta"].sel(county="AITKIN").values.flatten()
y = varying_intercept_slope_trace.posterior["sigma_b"].values.flatten()

diverging_mask = varying_intercept_slope_trace.sample_stats['diverging'].values.flatten().astype(bool)

go.Figure().add_trace(go.Scatter(
    x=x[~diverging_mask],
    y=y[~diverging_mask],
    mode='markers',
    name='Non-diverging',
    marker=dict(color='blue', size=8, opacity=0.2)
)).add_trace(go.Scatter(
    x=x[diverging_mask],
    y=y[diverging_mask],
    mode='markers',
    name='Diverging',
    marker=dict(color='orange', size=8, opacity=0.7)
)).update_layout(
    title="Neal's Funnel",
    xaxis_title='Beta (AITKIN county)',
    yaxis_title='Sigma_b',
    showlegend=True,
    yaxis=dict(range=[0, None])
)


When the group variance is small, this implies that the individual random slopes are themselves close to the group mean. This results in a *funnel*-shaped relationship between the samples of group variance and any of the slopes (particularly those with a smaller sample size). 

In itself, this is not a problem, since this is the behavior we expect. However, if the sampler is tuned for the wider (unconstrained) part of the parameter space, it has trouble in the areas of higher curvature. The consequence of this is that the neighborhood close to the lower bound of $\sigma_b$ is sampled poorly; indeed, in our chain it is not sampled at all below 0.1. In addtion, the sampler generates a lot of divergent samples. The result of this will be biased inference.

Now that we've spotted the problem, what can we do about it? The best way to deal with this issue is to reparameterize our model. Notice the random slopes in this version:

In [ ]:
with pm.Model(coords=coords) as varying_intercept_slope_noncentered:
    floor_idx = pm.Data("floor_idx", floor_measure, dims="obs_id")
    county_idx = pm.Data("county_idx", county, dims="obs_id")

    # Priors
    mu_a = pm.Normal("mu_a", mu=0.0, sigma=10.0)
    sigma_a = pm.Exponential("sigma_a", 5)

    # Non-centered random intercepts
    # Centered: a = pm.Normal('a', mu_a, sigma=sigma_a, shape=counties)
    z_a = pm.Normal("z_a", mu=0, sigma=1, dims="county")
    alpha = pm.Deterministic("alpha", mu_a + z_a * sigma_a, dims="county")

    mu_b = pm.Normal("mu_b", mu=0.0, sigma=10.0)
    sigma_b = pm.Exponential("sigma_b", 5)

    # Non-centered random slopes
    z_b = pm.Normal("z_b", mu=0, sigma=1, dims="county")
    beta = pm.Deterministic("beta", mu_b + z_b * sigma_b, dims="county")

    # Model error
    sigma_y = pm.Exponential("sigma_y", 5)

    # Expected value
    y_hat = alpha[county_idx] + beta[county_idx] * floor_idx

    # Data likelihood
    y_like = pm.Normal("y_like", mu=y_hat, sigma=sigma_y, observed=log_radon, dims="obs_id")

In [ ]:
pm.model_to_graphviz(varying_intercept_slope_noncentered)

This is a [**non-centered** parameterization](https://twiecki.io/blog/2017/02/08/bayesian-hierchical-non-centered/). By this, we mean that the random deviates are no longer explicitly modeled as being centered on $\mu_b$. Instead, they are independent standard normals $\upsilon$, which are then scaled by the appropriate value of $\sigma_b$, before being location-transformed by the mean.

This model samples much better.

In [ ]:
with varying_intercept_slope_noncentered:
    noncentered_trace = pm.sample(tune=3000, target_accept=0.95, random_seed=RANDOM_SEED)

Notice that the bottlenecks in the traces are gone.

In [ ]:
# Extract posterior samples for chain 0 using polars
sigma_b_df = noncentered_trace.posterior["sigma_b"].sel(chain=0).to_dataframe().reset_index()
beta_df = noncentered_trace.posterior["beta"].sel(chain=0).to_dataframe().reset_index()

fig, axs = plt.subplots(nrows=2)
axs[0].plot(sigma_b_df["sigma_b"].to_numpy(), alpha=0.5)
axs[0].set(ylabel="sigma_b")
axs[1].plot(beta_df["beta"].to_numpy(), alpha=0.5)
axs[1].set(ylabel="beta");

And correspondingly, the low end of the posterior distribution of the slope random effect variance can now be sampled efficiently.

In [ ]:
x = noncentered_trace.posterior["beta"].sel(county="AITKIN").values.flatten()
y = noncentered_trace.posterior["sigma_b"].values.flatten()

diverging_mask = noncentered_trace.sample_stats['diverging'].values.flatten().astype(bool)

go.Figure().add_trace(go.Scatter(
    x=x[~diverging_mask],
    y=y[~diverging_mask],
    mode='markers',
    name='Non-diverging',
    marker=dict(color='blue', size=8, opacity=0.2)
)).add_trace(go.Scatter(
    x=x[diverging_mask],
    y=y[diverging_mask],
    mode='markers',
    name='Diverging',
    marker=dict(color='orange', size=8, opacity=0.7)
)).update_layout(
    title="Neal's Funnel",
    xaxis_title='Beta (AITKIN county)',
    yaxis_title='Sigma_b',
    showlegend=True,
    yaxis=dict(range=[0, None])
)


As a result, we are now fully exploring the support of the posterior. This results in less bias in these parameters.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, constrained_layout=True)
az.plot_posterior(varying_intercept_slope_trace, var_names=["sigma_b"], ax=ax1)
az.plot_posterior(noncentered_trace, var_names=["sigma_b"], ax=ax2)
ax1.set_title("Centered (top) and non-centered (bottom)");

Notice that `sigma_b` now has a lot of density near zero, which would indicate that counties don't vary that much in their answer to the `floor` "treatment". 

This was the problem with the original parameterization: the sampler has difficulty with the geometry of the posterior distribution when the values of the slope random effects are so different for standard deviations very close to zero compared to when they are positive. However, even with the non-centered model the sampler is not that comfortable with `sigma_b`: in fact if you look at the estimates with `az.summary` you'll see that the number of effective samples is quite low for `sigma_b`.

Also note that `sigma_a` is not that big either -- i.e counties do differ in their baseline radon levels, but not by a lot. However we don't have that much of a problem to sample from this distribution because it's much narrower than `sigma_b` and doesn't get dangerously close to 0.

In [ ]:
az.summary(varying_intercept_slope_trace, var_names=["sigma_a", "sigma_b"])

To wrap up this model, let's plot the relationship between radon and floor for each county:

In [ ]:
xvals = xr.DataArray([0, 1], dims="Level", coords={"Level": ["Basement", "Floor"]})
post = noncentered_trace.posterior  # alias for readability
theta = (
    (post.alpha + post.beta * xvals).mean(dim=("chain", "draw")).to_dataset(name="Mean log radon")
)

_, ax = plt.subplots()
theta.plot.scatter(x="Level", y="Mean log radon", alpha=0.2, color="k", ax=ax)  # scatter
ax.plot(xvals, theta["Mean log radon"].T, "k-", alpha=0.2)
# add lines too
ax.set_title("MEAN LOG RADON BY COUNTY");

This, while both the intercept and the slope vary by county, there is far less variation in the slope. 

## Adding group-level predictors

A primary strength of multilevel models is the ability to handle predictors on multiple levels simultaneously. If we consider the varying-intercepts model above:

$$y_i = \alpha_{j[i]} + \beta x_{i} + \epsilon_i$$

we may, instead of a simple random effect to describe variation in the expected radon value, specify another regression model with a county-level covariate. Here, we use the county uranium reading $u_j$, which is thought to be related to radon levels:

$$\alpha_j = \gamma_0 + \gamma_1 u_j + \zeta_j$$

$$\zeta_j \sim N(0, \sigma_{\alpha}^2)$$

Thus, we are now incorporating a house-level predictor (floor or basement) as well as a county-level predictor (uranium).

Note that the model has both indicator variables for each county, plus a county-level covariate. In classical regression, this would result in collinearity. In a multilevel model, the partial pooling of the intercepts towards the expected value of the group-level linear model avoids this.

Group-level predictors also serve to reduce group-level variation, $\sigma_{\alpha}$ (here it would be the variation across counties, `sigma_a`). An important implication of this is that the group-level estimate induces stronger pooling -- by definition, a smaller $\sigma_{\alpha}$ means a stronger shrinkage of counties parameters towards the overall state mean. 

This is fairly straightforward to implement in PyMC -- we just add another level:

In [ ]:
with pm.Model(coords=coords) as hierarchical_intercept:
    # Priors
    sigma_a = pm.HalfCauchy("sigma_a", 5)

    # County uranium model
    gamma_0 = pm.Normal("gamma_0", mu=0.0, sigma=10.0)
    gamma_1 = pm.Normal("gamma_1", mu=0.0, sigma=10.0)

    # Uranium model for intercept
    mu_a = pm.Deterministic("mu_a", gamma_0 + gamma_1 * u)
    # County variation not explained by uranium
    epsilon_a = pm.Normal("epsilon_a", mu=0, sigma=1, dims="county")
    alpha = pm.Deterministic("alpha", mu_a + sigma_a * epsilon_a, dims="county")

    # Common slope
    beta = pm.Normal("beta", mu=0.0, sigma=10.0)

    # Model error
    sigma_y = pm.Uniform("sigma_y", lower=0, upper=100)

    # Expected value
    y_hat = alpha[county] + beta * floor_measure

    # Data likelihood
    y_like = pm.Normal("y_like", mu=y_hat, sigma=sigma_y, observed=log_radon)

In [ ]:
pm.model_to_graphviz(hierarchical_intercept)

Do you see the new level, with `sigma_a` and `gamma`, which is two-dimensional because it contains the linear model for `a_county`?

In [ ]:
with hierarchical_intercept:
    hierarchical_intercept_trace = pm.sample(tune=2000, random_seed=RANDOM_SEED)

In [ ]:
uranium = u
post = hierarchical_intercept_trace.posterior.assign_coords(uranium=uranium)
avg_a = post["mu_a"].mean(dim=("chain", "draw")).values[np.argsort(uranium)]
avg_a_county = post["alpha"].mean(dim=("chain", "draw"))
avg_a_county_hdi = az.hdi(post, var_names="alpha")["alpha"]

# Calculate HDI for the trend line (mu_a)
mu_a_hdi = az.hdi(post, var_names="mu_a")["mu_a"]
sorted_indices = np.argsort(uranium)

fig = go.Figure().add_trace(
    go.Scatter(
        x=np.concatenate([uranium[sorted_indices], uranium[sorted_indices][::-1]]),
        y=np.concatenate([mu_a_hdi.sel(hdi="lower").values[sorted_indices],

    mu_a_hdi.sel(hdi="higher").values[sorted_indices][::-1]]),
        fill='toself',
        fillcolor='rgba(128,128,128,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        showlegend=True,
        name='Mean intercept HPD',
        hoverinfo='skip'
    )
).add_trace(go.Scatter(
    x=uranium[sorted_indices],
    y=avg_a,
    mode='lines',
    line=dict(dash='dash', color='black', width=2),
    opacity=0.6,
    name='Mean intercept'
)).add_trace(go.Scatter(
    x=uranium,
    y=avg_a_county,
    mode='markers',
    marker=dict(color='teal', size=6, opacity=0.8),
    name='Mean county-intercept'
))

for i, u_val in enumerate(uranium):
    fig.add_shape(
        type="line",
        x0=u_val, x1=u_val,
        y0=avg_a_county_hdi.sel(hdi="lower").values[i],
        y1=avg_a_county_hdi.sel(hdi="higher").values[i],
        line=dict(color="orange", width=1.5),
        opacity=0.7
    )

fig.update_layout(
    xaxis_title="County-level uranium",
    yaxis_title="Intercept estimate",
    showlegend=True,
    plot_bgcolor='white',
    width=800,
    height=500
)

Uranium is indeed strongly associated with baseline radon levels in each county. The graph above shows the average relationship and its uncertainty: the baseline radon level in an average county as a function of uranium, as well as the 94% HPD of this radon level (dashed line and envelope). The blue points and orange bars represent the relationship between baseline radon and uranium, but now for each county. As you see, the uncertainty is bigger now, because it adds on top of the average uncertainty -- each county has its idyosyncracies after all.

If we compare the county-intercepts for this model with those of the partial-pooling model without a county-level covariate:The standard errors on the intercepts are narrower than for the partial-pooling model without a county-level covariate.

In [ ]:
ax = az.plot_forest(
    [varying_intercept_trace, hierarchical_intercept_trace],
    model_names=["W/t. county pred.", "With county pred."],
    var_names=["alpha"],
    combined=True,
    figsize=(6, 40),
    textsize=9,
    legend=False
)

# Clean up y-axis labels to show only county names (remove duplicates)
y_labels = [label.get_text() for label in ax[0].get_yticklabels()]
clean_labels = []
seen_counties = set()

for label in y_labels:
    if ':' in label:
        # Extract just the county name part (after the colon)
        county_name = label.split(':')[1].strip().replace('[', '').replace(']', '')
        if county_name not in seen_counties:
            clean_labels.append(county_name)
            seen_counties.add(county_name)
        else:
            clean_labels.append('')  # Empty string for duplicate
    else:
        clean_labels.append(label)

ax[0].set_yticklabels(clean_labels)
ax[0].set_ylabel("County")
ax[0].legend(["W/t. county pred.", "With county pred."], loc='upper center', bbox_to_anchor=(0.5, 1.02), ncol=2);

We see that the compatibility intervals are narrower for the model including the county-level covariate. This is expected, as the effect of a covariate is to reduce the variation in the outcome variable -- provided the covariate is of predictive value. More importantly, with this model we were able to squeeze even more information out of the data.

### Correlations among levels

In some instances, having predictors at multiple levels can reveal correlation between individual-level variables and group residuals. We can account for this by including the average of the individual predictors as a covariate in the model for the group intercept.

$$\alpha_j = \gamma_0 + \gamma_1 u_j + \gamma_2 \bar{x} + \zeta_j$$

These are broadly referred to as ***contextual effects***.

To add these effects to our model, let's create a new variable containing the mean of `floor` in each county and add that to our previous model:

In [ ]:
# Create new variable for mean of floor across counties
avg_floor_data = srrs_mn.group_by("county").agg(pl.col("floor").mean()).select("floor").to_numpy().flatten()

In [ ]:
with pm.Model(coords=coords) as contextual_effect:
    floor_idx = pm.Data("floor_idx", floor_measure)
    county_idx = pm.Data("county_idx", county)
    y = pm.Data("y", log_radon)

    # Priors
    sigma_a = pm.HalfCauchy("sigma_a", 5)

    # County uranium model for slope
    gamma = pm.Normal("gamma", mu=0.0, sigma=10, shape=3)

    # Uranium model for intercept
    mu_a = pm.Deterministic("mu_a", gamma[0] + gamma[1] * u + gamma[2] * avg_floor_data)

    # County variation not explained by uranium
    epsilon_a = pm.Normal("epsilon_a", mu=0, sigma=1, dims="county")
    alpha = pm.Deterministic("alpha", mu_a + sigma_a * epsilon_a)

    # Common slope
    beta = pm.Normal("beta", mu=0.0, sigma=10)

    # Model error
    sigma_y = pm.Uniform("sigma_y", lower=0, upper=100)

    # Expected value
    y_hat = alpha[county_idx] + beta * floor_idx

    # Data likelihood
    y_like = pm.Normal("y_like", mu=y_hat, sigma=sigma_y, observed=y)

In [ ]:
with contextual_effect:
    contextual_effect_trace = pm.sample(tune=2000, random_seed=RANDOM_SEED)

In [ ]:
az.summary(contextual_effect_trace, var_names="gamma", round_to=2)

So we might infer from this that counties with higher proportions of houses without basements tend to have higher baseline levels of radon. This seems to be new, as up to this point we saw that `floor` was *negatively* associated with radon levels. But remember this was at the household-level: radon tends to be higher in houses with basements. But at the county-level it seems that the less basements on average in the county, the more radon. So it's not that contradictory. What's more, the estimate for $\gamma_2$ is quite uncertain and overlaps with zero, so it's possible that the relationship is not that strong. And finally, let's note that $\gamma_2$ estimates something else than uranium's effect, as this is already taken into account by $\gamma_1$ -- it answers the question "once we know uranium level in the county, is there any value in learning about the proportion of houses without basements?".

All of this is to say that we shouldn't interpret this causally: there is no credible mechanism by which a basement (or absence thereof) *causes* radon emissions. More probably, our causal graph is missing something: a confounding variable, one that influences both basement construction and radon levels, is lurking somewhere in the dark... Perhaps is it the type of soil, which might influence what type of structures are built *and* the level of radon? Maybe adding this to our model would help with causal inference.

### Prediction

Gelman (2006) used cross-validation tests to check the prediction error of the unpooled, pooled, and partially-pooled models

**root mean squared cross-validation prediction errors**:

* unpooled = 0.86
* pooled = 0.84
* multilevel = 0.79

There are two types of prediction that can be made in a multilevel model:

1. a new individual within an existing group
2. a new individual within a new group

For example, if we wanted to make a prediction for a new house with no basement in St. Louis and Kanabec counties, we just need to sample from the radon model with the appropriate intercept.

That is, 

$$\tilde{y}_i \sim N(\alpha_{69} + \beta (x_i=1), \sigma_y^2)$$

Because we judiciously set the county index and floor values as shared variables earlier, we can modify them directly to the desired values (69 and 1 respectively) and sample corresponding posterior predictions, without having to redefine and recompile our model. Using the model just above:

In [ ]:
prediction_coords = {"obs_id": ["ST LOUIS", "KANABEC"]}
with contextual_effect:
    pm.set_data({"county_idx": np.array([69, 31]), "floor_idx": np.array([1, 1]), "y": np.ones(2)})
    stl_pred = pm.sample_posterior_predictive(contextual_effect_trace.posterior)

contextual_effect_trace.extend(stl_pred)

In [ ]:
az.plot_posterior(contextual_effect_trace, group="posterior_predictive");

Prediction for a house within a new county is a little trickier. It is actually easier to create a new model to work with, **but use the trace from the original model for posterior predictive sampling**. 

How can this work?

First, consider how posterior predictive sampling works in PyMC: samples are drawn not from the distributions themselves, but from the set of samples in the trace. Therefore, we can take the trace from the original model, and use it to sample posterior predictions from a new model that has the same variables.

The variables in the new model need only have the same name as the original -- to reinforce this, I will use `pm.Flat` variables as placeholders in this example. The only variables we actually need are the ones that need to be resampled for a new county.

We don't even need `Data` here; we can use raw data, since we are just creating this model to get posterior predictions for houses in this notional new county.

In [ ]:
with pm.Model() as new_county_house:

    # New data
    u_new = np.array([-0.2, 0.3])
    xbar = np.array([0.5, 0.8])
    floor_idx = np.array([1, 0])
    
    # Placeholders for variables already in the trace
    sigma_a = pm.Flat('sigma_a')
    gamma = pm.Flat('gamma', shape=3)
    beta = pm.Flat('beta')
    sigma_y = pm.Flat('sigma_y')

    # Calculate new county expected value
    mu_a_new = pm.Deterministic('mu_a_new', gamma[0] + gamma[1]*u_new + gamma[2]*xbar)

    # Sample from the county intercept distribution
    mu_new = pm.Normal('mu_new', mu_a_new, sigma_a)

    # Expected value for houses in new county
    y_hat_new = mu_new + beta * floor_idx
    
    y_new = pm.Normal('y_new', mu=y_hat_new, sigma=sigma_y)

    pp_new = pm.sample_posterior_predictive(contextual_effect_trace, var_names=["y_new"])

In [ ]:
az.plot_posterior(pp_new, group='posterior_predictive');

## Benefits of Multilevel Models

- Accounting for natural hierarchical structure of observational data.

- Estimation of coefficients for (under-represented) groups.

- Incorporating individual- and group-level information when estimating group-level coefficients.

- Allowing for variation among individual-level coefficients across groups.

As an alternative approach to hierarchical modeling for this problem, check out a [geospatial approach](https://www.pymc-labs.io/blog-posts/spatial-gaussian-process-01/) to modeling radon levels.

---
## References

Gelman, A., & Hill, J. (2006). Data Analysis Using Regression and Multilevel/Hierarchical Models (1st ed.). Cambridge University Press.

Betancourt, M. J., & Girolami, M. (2013). Hamiltonian Monte Carlo for Hierarchical Models.

Gelman, A. (2006). Multilevel (Hierarchical) modeling: what it can and cannot do. Technometrics, 48(3), 432–435.

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w